<a href="https://colab.research.google.com/github/Yukselendincer/datasceinceproject/blob/main/PerakendeCOSO_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Klasik yöntemde denetçiler Excel'de örneklem seçip (sampling) manuel kontrol yapar. Ben ise COSO prensiplerine uygun Python scriptleri geliştiriyorum.

Örneğin, hazırladığım bir projede; 5.000 satırlık satış verisini saniyeler içinde tarayıp, şirket politikası olan %30 iskonto limitini aşanları ve istatistiksel olarak imkansız iade oranına sahip personelleri nokta atışı tespit eden bir algoritma yazdım. Bu sayede reaktif denetimden (olay olduktan sonra), proaktif (sürekli izleme) denetime geçişi sağlıyorum. Skechers'ın mağaza ağındaki veriyi bu şekilde işleyerek riskleri minimize edebilirim.

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# --- BÖLÜM 1: SENTETİK VERİ ÜRETİMİ (Simülasyon) ---
# Skechers mağazalarındaki günlük operasyonu taklit ediyoruz.
def generate_synthetic_data(num_records=5000):
    np.random.seed(42) # Tekrarlanabilir sonuçlar için
    random.seed(42)

    dates = [datetime(2024, 1, 1) + timedelta(days=x) for x in range(30)]
    stores = [f'Magaza_{i}' for i in range(1, 21)] # 20 Mağaza
    cashiers = [f'Sicil_{i}' for i in range(1001, 1051)] # 50 Kasiyer
    products = ['Ayakkabı', 'Tekstil', 'Aksesuar']

    data = []

    for _ in range(num_records):
        date = random.choice(dates)
        store = random.choice(stores)
        cashier = random.choice(cashiers)
        category = random.choice(products)
        amount = round(random.uniform(1500, 8000), 2) # Satış Tutarı

        # Risk Senaryosu Enjekte Ediyoruz:
        # %3 ihtimalle bir kasiyer "Suistimal" yapıyor (Yetkisiz yüksek iskonto)
        risk_factor = random.random()

        if risk_factor < 0.03:
            # Riskli İşlem: %40-%70 arası yetkisiz iskonto
            discount_rate = round(random.uniform(0.40, 0.70), 2)
            is_refund = 0
        else:
            # Normal İşlem: %0-%25 arası kampanya iskontosu
            discount_rate = round(random.uniform(0, 0.25), 2)
            # %5 ihtimalle normal bir iade işlemi
            is_refund = 1 if random.random() < 0.05 else 0

        data.append([date, store, cashier, category, amount, discount_rate, is_refund])

    columns = ['Tarih', 'Magaza_Kodu', 'Personel_Sicil', 'Kategori', 'Tutar', 'Iskonto_Orani', 'Iade_Mi']
    return pd.DataFrame(data, columns=columns)

# Veriyi Üret
df = generate_synthetic_data()

# --- BÖLÜM 2: COSO KONTROL AKTİVİTELERİ (Denetim Mantığı) ---

# KONTROL 1: Politika Limit Kontrolü (Compliance)
# "Mağaza müdürleri onaysız en fazla %30 iskonto yapabilir." kuralını denetliyoruz.
def detect_discount_abuse(row):
    LIMIT = 0.30
    if row['Iskonto_Orani'] > LIMIT:
        return 'BULGU: Yetki Aşımı İskonto'
    return 'Normal'

# KONTROL 2: İstatistiksel Anomali Tespiti (Analytics)
# Personel bazlı iade oranlarını analiz edip, ortalamadan 3 standart sapma sapanları buluyoruz.
def detect_refund_anomalies(df):
    stats = df.groupby('Personel_Sicil').agg({'Iade_Mi': ['count', 'mean']})
    stats.columns = ['Islem_Adedi', 'Iade_Orani']

    # Eşik Değer Hesaplama (Mean + 3 Std Dev)
    threshold = stats['Iade_Orani'].mean() + (3 * stats['Iade_Orani'].std())

    suspicious_staff = stats[stats['Iade_Orani'] > threshold].index.tolist()
    return suspicious_staff, threshold

# Denetimi Çalıştır
df['Denetim_Sonucu'] = df.apply(detect_discount_abuse, axis=1)
supheli_personeller, esik_deger = detect_refund_anomalies(df)

# İade anomalilerini de ana tabloya işaretle
df.loc[df['Personel_Sicil'].isin(supheli_personeller) & (df['Iade_Mi']==1), 'Denetim_Sonucu'] = 'BULGU: Şüpheli İade Aktivitesi'

# --- BÖLÜM 3: RAPORLAMA VE İZLEME (Reporting) ---

bulgular = df[df['Denetim_Sonucu'] != 'Normal']

print(f"--- İÇ DENETİM OTOMASYON RAPORU ---")
print(f"Taranan Toplam İşlem: {len(df)}")
print(f"Tespit Edilen Riskli İşlem: {len(bulgular)}")
print(f"Risk Oranı: %{(len(bulgular)/len(df))*100:.2f}")
print("-" * 40)
print("\n[Risk 1] İskonto Limit Aşımları (Mağaza Bazlı Özeti):")
print(bulgular[bulgular['Denetim_Sonucu'] == 'BULGU: Yetki Aşımı İskonto']['Magaza_Kodu'].value_counts().head())

print("-" * 40)
print(f"\n[Risk 2] İade Anomalisi (Eşik Değer: %{esik_deger*100:.2f})")
print(f"Tespit Edilen Şüpheli Personeller: {supheli_personeller}")

--- İÇ DENETİM OTOMASYON RAPORU ---
Taranan Toplam İşlem: 5000
Tespit Edilen Riskli İşlem: 145
Risk Oranı: %2.90
----------------------------------------

[Risk 1] İskonto Limit Aşımları (Mağaza Bazlı Özeti):
Magaza_Kodu
Magaza_14    12
Magaza_15    11
Magaza_13    11
Magaza_4      9
Magaza_17     9
Name: count, dtype: int64
----------------------------------------

[Risk 2] İade Anomalisi (Eşik Değer: %10.53)
Tespit Edilen Şüpheli Personeller: []


Şirkette satın alma müdürlerinin onay limiti 50.000 TL'dir. Bazı çalışanlar, 90.000 TL'lik bir alımı onaya takılmadan geçirmek için iki adet 45.000 TL'lik faturaya bölüyorlar (Structuring Fraud). Ayrıca veritabanında mükerrer tedarikçi kayıtları (Örn: 'ABC Lojistik' ve 'ABC Lojistik A.Ş.') üzerinden çifte ödeme riski var.

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# --- BÖLÜM 1: SENTETİK SATIN ALMA VERİSİ (Simülasyon) ---
def generate_procurement_data(num_invoices=2000):
    np.random.seed(101)
    random.seed(101)

    vendors = [f'Tedarikci_{i}' for i in range(1, 51)] # 50 farklı tedarikçi
    dates = [datetime(2024, 1, 1) + timedelta(days=x) for x in range(90)]

    data = []

    for i in range(num_invoices):
        invoice_id = f'INV-{10000+i}'
        vendor = random.choice(vendors)
        date = random.choice(dates)

        # Normal Fatura Tutarları (Log-normal dağılım - doğal finansal veri gibi)
        amount = round(np.random.lognormal(mean=8, sigma=1), 2)

        data.append([invoice_id, vendor, date, amount, 'Normal'])

    # --- HİLE ENJEKSİYONU (Fraud Injection) ---

    # Senaryo 1: Onay Limitinden Kaçınma (Split Invoices)
    # 50.000 TL limitini aşmamak için 48.500 TL'lik ardışık faturalar
    fraud_vendor = 'Tedarikci_X_Hileli'
    fraud_date = datetime(2024, 2, 15)

    # Aynı gün, aynı tedarikçiden, limitin hemen altında 3 fatura
    data.append(['INV-FRAUD-1', fraud_vendor, fraud_date, 49500.00, 'Risk: Split Invoice'])
    data.append(['INV-FRAUD-2', fraud_vendor, fraud_date, 48900.00, 'Risk: Split Invoice'])
    data.append(['INV-FRAUD-3', fraud_vendor, fraud_date, 49100.00, 'Risk: Split Invoice'])

    # Senaryo 2: Benford Yasasına Aykırı Rakamlar
    # İnsan eliyle uydurulmuş rakamlar genellikle 1 ile başlamaz, rastgele dağılmaz.
    # 9 ile başlayan şüpheli tutarlar ekleyelim.
    for k in range(10):
        data.append([f'INV-BENF-{k}', 'Tedarikci_Y_Supheli', random.choice(dates), 9990.00, 'Risk: Benford Anomaly'])

    df = pd.DataFrame(data, columns=['Fatura_No', 'Tedarikci', 'Tarih', 'Tutar', 'Gercek_Durum'])
    return df

df = generate_procurement_data()

# --- BÖLÜM 2: DENETİM ALGORİTMALARI ---

# 1. Benford Yasası Analizi (İlk Basamak Analizi)
# Finansal verilerde ilk basamağın '1' olma ihtimali %30.1'dir. '9' olma ihtimali %4.6'dır.
# Eğer '9' ile başlayanlar çok fazlaysa, veriler manipüle edilmiş olabilir.
def check_benford(df):
    # İlk basamağı al
    df['Ilk_Basamak'] = df['Tutar'].astype(str).str[0].astype(int)
    # 0 ve negatifleri ele (sadece 1-9 arası)
    benford_df = df[df['Ilk_Basamak'] > 0]

    digit_counts = benford_df['Ilk_Basamak'].value_counts(normalize=True).sort_index()

    # Benford Referans Değerleri (Teorik)
    benford_ref = {1: 0.301, 2: 0.176, 3: 0.125, 4: 0.097, 5: 0.079, 6: 0.067, 7: 0.058, 8: 0.051, 9: 0.046}

    results = []
    print("\n--- BENFORD YASASI ANALİZİ ---")
    print(f"{'Basamak':<10} {'Gerçekleşen':<15} {'Benford Teorik':<15} {'Sapma'}")

    for digit, theoretical in benford_ref.items():
        actual = digit_counts.get(digit, 0)
        diff = abs(actual - theoretical)
        print(f"{digit:<10} %{actual*100:.1f}{' ':<10} %{theoretical*100:.1f}{' ':<10} %{diff*100:.1f}")

        if diff > 0.05: # %5'ten fazla sapma varsa uyarı ver
            print(f"  >>> UYARI: {digit} rakamında anomali var!")

# 2. Bölünmüş Fatura (Split Invoice) Tespiti
# Aynı gün, aynı tedarikçiden, toplamı onay limitini (50k) aşan ama tek tek altında kalan faturalar.
def detect_split_invoices(df):
    APPROVAL_LIMIT = 50000

    # Aynı gün ve aynı tedarikçiye göre grupla
    grouped = df.groupby(['Tedarikci', 'Tarih']).agg({'Tutar': ['sum', 'count', list], 'Fatura_No': list})
    grouped.columns = ['Gunluk_Toplam', 'Fatura_Adedi', 'Tutarlar', 'Fatura_Nolar']

    # Kriter: Günlük toplam limiti aşıyor AMA fatura adedi > 1 (yani bölünmüş)
    potential_fraud = grouped[(grouped['Gunluk_Toplam'] > APPROVAL_LIMIT) & (grouped['Fatura_Adedi'] > 1)]

    return potential_fraud

# --- BÖLÜM 3: ÇALIŞTIRMA VE RAPORLAMA ---

check_benford(df)

split_frauds = detect_split_invoices(df)

print("\n" + "="*50)
print("--- SPLIT INVOICE (BÖLÜNMÜŞ FATURA) BULGULARI ---")
if not split_frauds.empty:
    for index, row in split_frauds.iterrows():
        print(f"\nTedarikçi: {index[0]}")
        print(f"Tarih: {index[1].date()}")
        print(f"Günlük Toplam: {row['Gunluk_Toplam']:.2f} TL (Limit: 50.000 TL)")
        print(f"Fatura Detayları: {row['Tutarlar']}")
        print(f"Şüpheli Faturalar: {row['Fatura_Nolar']}")
        print("YORUM: Onay limitinden kaçınmak için faturalar bölünmüş olabilir.")
else:
    print("Bulgu yok.")


--- BENFORD YASASI ANALİZİ ---
Basamak    Gerçekleşen     Benford Teorik  Sapma
1          %30.5           %30.1           %0.4
2          %18.4           %17.6           %0.8
3          %12.2           %12.5           %0.3
4          %9.8           %9.7           %0.1
5          %7.3           %7.9           %0.6
6          %6.9           %6.7           %0.2
7          %5.5           %5.8           %0.3
8          %5.2           %5.1           %0.1
9          %4.3           %4.6           %0.3

--- SPLIT INVOICE (BÖLÜNMÜŞ FATURA) BULGULARI ---

Tedarikçi: Tedarikci_13
Tarih: 2024-03-12
Günlük Toplam: 71864.91 TL (Limit: 50.000 TL)
Fatura Detayları: [68506.96, 3357.95]
Şüpheli Faturalar: ['INV-11305', 'INV-11958']
YORUM: Onay limitinden kaçınmak için faturalar bölünmüş olabilir.

Tedarikçi: Tedarikci_22
Tarih: 2024-02-21
Günlük Toplam: 216699.95 TL (Limit: 50.000 TL)
Fatura Detayları: [5487.14, 211212.81]
Şüpheli Faturalar: ['INV-10735', 'INV-11996']
YORUM: Onay limitinden kaçınmak iç

Risk Değerlendirme (Risk Assessment):

"Şirketin nakit çıkışlarındaki en büyük risk, onay mekanizmalarının delinmesi (bypass) riskidir. Python kodum, bu riski 'Split Invoice' algoritmasıyla doğrudan hedefler."

Kontrol Faaliyetleri (Control Activities):

"Otomatik bir kontrol mekanizması (Automated Control) tasarladım. Bu kod, manuel kontrollerin yakalayamayacağı, farklı zamanlarda girilmiş ama ilişkili faturaları yakalar."

İzleme (Monitoring):

"Bu analiz, yıl sonunda değil, her ay veya her hafta çalıştırılarak 'Sürekli Denetim' (Continuous Audit) sağlar."